In [55]:
import os
import xml.etree.ElementTree as ET
import numpy as np

In [53]:
import pandas as pd
import os
import google.oauth2.credentials
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow

ModuleNotFoundError: No module named 'google'

In [12]:

CLIENT_SECRETS_FILE = "client_secret.json"
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'
def get_authenticated_service():
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
    credentials = flow.run_console()
    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)
# Remove keyword arguments that are not set
def remove_empty_kwargs(**kwargs):
    good_kwargs = {}
    if kwargs is not None:
        for key, value in kwargs.items():
            if value:
                good_kwargs[key] = value
    return good_kwargs

In [15]:
client = get_authenticated_service()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=692697523900-m6jnb73lek6udl104gvou0joh8t0771q.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=O2rgDFrfJ5kVY062gbALmtf5oW3yKz&prompt=consent&access_type=offline
Enter the authorization code: 4/0AEtq3xJseFOZUi2wEx63TZEGh4StR31UgrWxjuRbZDDaHq9cMuUe7Y


In [16]:
def youtube_keyword(client, **kwargs):    
    kwargs = remove_empty_kwargs(**kwargs)
    response = client.search().list(
        **kwargs
        ).execute()    
    return response

In [35]:
def youtube_search (criteria,max_res):   
    #create lists and empty dataframe
    titles = []
    videoIds = []
    channelIds = []
    resp_df = pd.DataFrame()
    
    while len(titles) < max_res:
        token = None
        response = youtube_keyword(client,
                        part='id,snippet',
                        maxResults=50,
                        q=criteria,
                        videoCaption='closedCaption',
                        type='video', 
                        #videoDuration='long',
                        #I18nRegion='de',
                        pageToken=token,
                        relevanceLanguage='de',
                        regionCode='de') 
                                         
        for item in response['items']:        
            titles.append(item['snippet']['title'])
            channelIds.append(item['snippet']['channelTitle'])
            videoIds.append(item['id']['videoId'])
        
        token = response['nextPageToken']
        
    resp_df['title'] = titles
    resp_df['channelId'] = channelIds
    resp_df['videoId'] = videoIds
    resp_df['subject'] = criteria
    
    return resp_df

In [37]:
Gates_Videos = youtube_search('[Impfen+Bill+Gates]',10)

HttpError: <HttpError 403 when requesting https://www.googleapis.com/youtube/v3/search?part=id%2Csnippet&maxResults=50&q=%5BImpfen%2BBill%2BGates%5D&videoCaption=closedCaption&type=video&relevanceLanguage=de&regionCode=de&alt=json returned "Daily Limit Exceeded. The quota will be reset at midnight Pacific Time (PT). You may monitor your quota usage and adjust limits in the API Console: https://console.developers.google.com/apis/api/youtube.googleapis.com/quotas?project=692697523900">

In [7]:
Gates_Videos.loc[0:40]

,Unnamed: 0,title,channelId,videoId,subject
0,0,Bill Gates will die Welt impfen,veggietation,AhuU_CG9v80,[Impfen+Gates]
1,1,Bill Gates und Bevölkerungskontrolle,veggietation,YWjkYM8slFU,[Impfen+Gates]
2,2,Wie Bill Gates das Weltgesundheitssystem monop...,veggietation,QVYj1HQyPg0,[Impfen+Gates]
3,3,Innovating to zero! | Bill Gates,TED,JaF-fq2Zn7I,[Impfen+Gates]
4,4,Bill Gates on Fighting Coronavirus | The Daily...,The Daily Show with Trevor Noah,iyFT8qXcOrM,[Impfen+Gates]
5,5,Dok 1 - &quot;Impfen? Nein danke!&quot;,Christian Tieber,8nH_0sauWxk,[Impfen+Gates]
6,6,Bill Gates at MIT 2010 - &quot;Giving Back: Fi...,MIT Video Productions,bfIqbOi9RCQ,[Impfen+Gates]
7,7,Jens Spahn im Interview mit YouTubern bei Brea...,Bundesministerium für Gesundheit,RsP9AuKKu20,[Impfen+Gates]
8,8,Online-Bürgersprechstunde mit Ministerpräsiden...,FreistaatSachsen,O8iMjdBFY3A,[Impfen+Gates]
9,9,Corona: die krassesten Thesen und Lügen | STRG_F,STRG_F,3duErFbfFM0,[Impfen+Gates]


In [57]:
import pandas as pd
vid_ids = pd.read_csv('idlst.csv')

In [61]:
vid_ids = vid_ids.loc[:,'0'].tolist()

In [58]:
len(vid_ids)

540

In [59]:
def get_all_ccs(videos):
    base_url = 'https://www.youtube.com/watch?v='
    #lang="de"
    for vid in videos:
        url = base_url + vid
        cmd = ["youtube-dl","--skip-download","--write-sub",url]
        os.system(" ".join(cmd))

In [37]:
len(set(vids))

NameError: name 'vids' is not defined

#### Using the above created function

In [62]:
get_all_ccs(vid_ids)

In [15]:
sub_vids

['sYT0A26wsxQ', '72plsUzUjP4']

In [44]:
vid_ids

['DyIH3O0NRsM',
 '43LiB6Ynmmw',
 'BX1_2j-QQuk',
 'QtNn2BE_6rE',
 'lz4dDKSizGg',
 'oXhUl4jhiGU',
 'KkPmlYZ_0_g',
 'bRD4BGec0wU',
 'O_tF0qAf2sY',
 '07HNYmIAuwE',
 'id1ryXAgygE',
 'csozbIUq3wY',
 'jIM5o-GdGKw',
 'Gnh4GBzgPHM',
 'uB-d0qCLXfA',
 'bKU8qKzaK_8',
 'tBNNPdyMshE',
 '_dgc1I-64QU',
 'jLbJayQygzw',
 'Y6OQ15nzoFc',
 'U-onOPOIV60',
 'jvLcWSYGyJ8',
 'DSvhPnUgyz8',
 'OAnLTiBIRts',
 'gmhbeb0dVFA',
 'h_oSEqDXYkI',
 'lCkIjYhAdHA',
 'NtdnddEXWlE',
 'wUtLKJUxBPA',
 'xrNzbrYuCOE',
 'PJNfJOdQ_eI',
 'v0UdgSEs6LI',
 'BWyTeZlx55w',
 'QIf4idTBz7E',
 'ETAnfJiAeq8',
 'YEbQgKIXWVc',
 'YWjkYM8slFU',
 'c7zJSRTcwrY',
 'il7RJ9BmGOg',
 'iqB8Quc8A9g',
 'sw4szXqz0Ro',
 '4R8CB0QqTA8',
 'WUF6c1uAlfw',
 'MgnPFncOE8o',
 'fpZGzeFTBbY',
 'aN_BO4Sez-Q',
 'Q2VPN8Yr1BU',
 'igx86PoU7v8',
 'I1aZzNlfq4g',
 'zGrry9a6KfM',
 'hXR2KKyQt0Q',
 'nVpWXB5IQLw',
 'CgYTeXtQbds',
 'K5SPfG2vErQ',
 'kUuGqxUzPKY',
 '8KzJ5cH5_tg',
 'CAVEkWUOaIE',
 'qN6bwZE0v-c',
 'mZ32rLrL2ag',
 'mX51utvnj9E',
 'oef5YexoLck',
 'AhuU_CG9v80',
 '8-etkt

In [4]:
vids = [item[0] for item in Gates_Videos.values.tolist()]

In [66]:
filenames_vtt

['¿Por qué nadie habla de Jeffrey Epstein, Bill Gates & la vacuna del coronavirus-QYEz_0hhrVI.es-ES.vtt',
 'Bill Gates talks about Paul Allen during speech at new UW computer science building-S01f6MuBCtE.en.vtt',
 'A Global Cashless Society has been the Goal of the Bill Gates Foundation Since 2012-xrNzbrYuCOE.en-GB.vtt',
 "Bill and Melinda Gates' 2014 Stanford Commencement Address-wug9n5Atk8c.en.vtt",
 'Bill Gates on Finding a Vaccine for COVID-19, the Economy, and Returning to ‘Normal Life’-5oEcxMfwJnw.en.vtt',
 'Bill Gates krijgt zijn vermogen niet weggegeven • Z zoekt uit---kmm2el4tE.nl.vtt',
 "Bill Gates's Heroes in the Field - Nandan Nilekani-q_I5oj9LEh8.en.vtt",
 'Bill Gates On Changing Behaviors To Fight Covid-19 _ The Last Word _ MSNBC-aMU5AlwlaKE.en.vtt',
 'RAMALAN BILL GATES...!!! Bill gates sudah meramal adanya SUPER VIRUS dari CHINA-c4STlP-yHcE.en.vtt',
 'Bill Gates meet Mark Zuckerberg Finally heading back to school Memory-4anZ2tY6J5g.zh-CN.vtt',
 'Bill Gates on the good n

In [93]:
filenames_vtt = [os.fsdecode(file) for file in os.listdir(os.getcwd()) if os.fsdecode(file).endswith(".vtt")]
#Check file names


In [94]:
filenames_vtt[0]

'¿Por qué nadie habla de Jeffrey Epstein, Bill Gates & la vacuna del coronavirus-QYEz_0hhrVI.es-ES.vtt'

#### Converting vtt to csv

In [67]:
import webvtt
def convert_vtt(filenames):    
    #create an assets folder if one does not yet exist
    if os.path.isdir('{}/assets'.format(os.getcwd())) == False:
        os.makedirs('assets')
    #extract the text and times from the vtt file
    for file in filenames:
        try:
            captions = webvtt.read(file)
            text_time = pd.DataFrame()
            text_time['text'] = [caption.text for caption in captions]
            text_time['start'] = [caption.start for caption in captions]
            text_time['stop'] = [caption.end for caption in captions]
            text_time.to_csv('assets/{}.csv'.format(file[:-4]),index=False) #-4 to remove '.vtt'
        except:
            captions = webvtt.read(file)
            text_time = pd.DataFrame()
            text_time['text'] = None
            text_time['start'] = None
            text_time['stop'] = None
            text_time.to_csv('assets/{}.csv'.format(file[:-4]),index=False)
            print('err')
            pass
        #remove files from local drive
        #os.remove(file)

In [46]:
text_time = pd.DataFrame()
text_time['text'] = None
text_time['start'] = None
text_time['stop'] = None

In [48]:
text_time.to_csv('sometest.csv')

In [56]:
np.where("Bill Gates Wants to Save the World" in filenames_vtt)

(array([], dtype=int64),)

In [51]:
filenames_vtt[]

['Robert F. Kennedy jr. - „Bill Gates’ globalistisch-diktatorische Impf-Agenda“ _ 22.04.2020 _ kla.tv-gYIpxGJwqEA.de.vtt',
 'Bill Gates chce zabić (prawie) wszystkie komary-LV_BdSKSFyo.pl.vtt',
 'VERIFY - Bill Gates and microchipping-1-n3HLUJEGE.en.vtt',
 'A Conversation with the 2017 Washington State Teacher of the Year-8Ygy1VpV3KA.en.vtt',
 'Bill Gates Compra Iate Movido Hidrogênio Por R$2,456,664,000 Bilhões-H6piv8STOnk.en.vtt',
 '5 books worth reading this summer-pwGbwYAfSmg.en.vtt',
 'bill gates protes pemerintah...!!! Amerika kalah dengan asia cara atasi CORONA-ZpbHJs1JHV8.en.vtt',
 'Jangan Contoh Bill Gates!-UyZeDjp3D5c.en.vtt',
 'Bill Gates’s Heroes in the Field - Mandy Manning-WOtu1S6xiAo.en.vtt',
 '7 frases de BILL GATES para MUDAR SUA VIDA!-4Lh6D9r0u4g.pt-BR.vtt',
 'Who is the most enlightened leader Bill Gates, Jeff Bezos, Jack Ma, Steve Jobs, Elon Musk-2FjlxNySaHE.de.vtt',
 'Meet Bill Gates-DSvhPnUgyz8.en.vtt',
 'Bill Gates & Warren Buffett - DO IT RIGHT NOW ~ A LIFE CHANG

In [68]:
#call the function
convert_vtt(filenames_vtt)

In [102]:
csv_files = [os.fsdecode(file) for file in os.listdir(os.getcwd()+'/assets') if os.fsdecode(file).endswith('.csv')]

In [103]:
csv_files[0]

'BeethovenasBillGates-JoseBowenatTEDxSMU2011-Jl3gnP4uG2o.en.csv'

In [72]:
len(vid_ids)

540

In [71]:
len(filenames_vtt)

536

In [70]:
len(csv_files)

536

In [104]:
path = 'assets/'
for filename in csv_files:
    os.rename(os.path.join(path, filename), os.path.join(path, filename.replace(' ', '')))
#to verify the results
clean_csv = [os.fsdecode(file) for file in os.listdir(os.getcwd()+'/assets')]
clean_csv[0]

'BeethovenasBillGates-JoseBowenatTEDxSMU2011-Jl3gnP4uG2o.en.csv'

In [105]:
#extract the text and videoid
vidText = []
csv_vidid = []

for file in clean_csv:
   df = pd.read_csv(path+file)
   text = " ".join(df.text)
   vidText.append(text)
   csv_vidid.append(file[-18:-7])

In [108]:
csv_vidid

['Jl3gnP4uG2o',
 'BX1_2j-QQuk',
 'XJn_4cPT6nE',
 'OAx_6-wdslM',
 'gTjXTRz4dDw',
 '_0QdI0rVoHA',
 'bIuhCUxpPsQ',
 'aVtD0swv94c',
 'zwGdnCNi8Ss',
 'nddEXWlE.de',
 'ogMEpsGh9Yo',
 'xtY4ThJ9i9E',
 '3tCUWbhyiig',
 'pQocN_c2uLI',
 'OzsjvQAk-eA',
 '6usVX6BJYEk',
 'WGmOePtI.en',
 'SWvcwYsMDew',
 'p0pgrq-DXCY',
 'vqqVbU0nkOE',
 'zmtxU8q6ohA',
 'id1ryXAgygE',
 'WOtu1S6xiAo',
 'LaAPJdO3dgA',
 'r9r4c1d8kSw',
 'jiUKpX09zo4',
 'ug1pb8rkI0s',
 'NKntTAGs.pt',
 'Z2tY6J5g.zh',
 'ZpbHJs1JHV8',
 'TpqdERCZrSU',
 '2CC5ejfjFq4',
 '7DmpncBDibA',
 'bRD4BGec0wU',
 'gWWDexIJ8mM',
 'hXR2KKyQt0Q',
 '_uvzKzWpRNI',
 'l3a22TP5r58',
 '1MbHoM3Izwo',
 'PeTOkUr1aaY',
 '4XVzSTN4Cv8',
 'k-gRgaAgTyw',
 'Kfdvg9KPwcA',
 '1BEjH0TCnCc',
 'i0ximl5ynZ8',
 'q8N_jQR712Q',
 'iKY4zhkwUnA',
 'kkRBZ5wg.en',
 'AVOyrj48MIM',
 'WDUjIcmHBH4',
 'z8imouImljY',
 '3EBBRsJA.en',
 '8-etktBzFos',
 'ETAnfJiAeq8',
 'aMU5AlwlaKE',
 'mdiQtv4UFk4',
 '5urZ9rrUVGM',
 '6YP7kYdn7dU',
 'MNfsLMDcGZ8',
 'fGYz5SszZ74',
 '8QQwcmJyO8E',
 'rZdiH9VKq70',
 'v0UdgS

In [107]:
len(csv_vidid)

536

In [106]:
len(vidText)

536

In [114]:
texts = pd.DataFrame({'vidID': csv_vidid, 'vidText': vidText})

In [116]:
texts.to_csv('video_texts.csv')

In [79]:
titles = pd.DataFrame(csv_files)

In [85]:
vtxt

,0
0,Transcriber: Marcia de Brito\nReviewer: Ariana...
1,"Bonjour. Je suis Bill Gates, le président de M..."
2,It's important to me to connect with the\npeop...
3,"[music] Everywhere you look, computers are cha..."
4,"For a good read this summer, try these five\nb..."
5,"Hi everyone, it's Jerry. Ever since Magnus Car..."
6,UPDATE FOR YOU ON EYEWITNESS NEWS AT SIX. (MM)...
7,(Image source: Wikimedia Commons / Kees de\nVo...
8,"Da Vinci lived from 1452 to 1519, so it’s the ..."
9,Wer ist der einflussreichste Mann der Welt?\nD...


In [82]:
titles.join(vtxt)

ValueError: columns overlap but no suffix specified: RangeIndex(start=0, stop=1, step=1)

In [81]:
vtxt = pd.DataFrame(vidText)
vID = pd.DataFrame(csv_vidid)


In [28]:
len(set(csv_vidid))

574

In [29]:
len(set(vidText))

572

In [21]:
vtxt.join(vID)

ValueError: columns overlap but no suffix specified: RangeIndex(start=0, stop=1, step=1)

In [17]:
vtxt.to_csv('vidTexts.csv', index = False)
vID.to_csv('vidID.csv', index = False)

In [58]:
vidText[12]


"of of of of this of of of of of of of of house of blues of of\nof a of of of of of of the of an aunt of force and not make the the same as the end of the facts of this as a foundation of Santa has fashioned a variant of that, and he has this have to have come to the forefront ruled no\nplace for common sense to the\nsuns the info as 2020 bodies might dish out which can only mean one thing programme you are watching the Conejo,\nwhich discusses policies social\nissues my name is my de my name is my de chat through the prism of common\nsense my name is my de Charles my name is mine de chacal my name is my name is mine de chacal de Challes de Challes you are de Challes you are we de Challes you are one de Challes you are watch de Challes you are we see take it with us in the de Challes you are watching on to resets de Challes you are watching on\nparle de prison what it all these de Challes you are watching on\nparle de prison oui the crusty old office says you\nknow de Challes you are w

In [50]:
Gates_Videos.to_csv('GatesVids.csv')

In [3]:
Gates_Videos = pd.read_csv("GatesVids.csv")

In [4]:
Gates_Videos = pd.read_csv("idlst.csv")